In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import preprocessing 

In [2]:
data_url='http://archive.ics.uci.edu/ml/machine-learning-databases/postoperative-patient-data/post-operative.data'
names = ['L-CORE', 'L-SURF', 'L-02', 'L-BP', 'SURF-STBL', 'CORE-STBL', 'BP-STBL', 'COMFPRT', 'decision']
df = pd.read_csv(data_url, names=names)
df[:5]

,L-CORE,L-SURF,L-02,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFPRT,decision
0,mid,low,excellent,mid,stable,stable,stable,15,A
1,mid,high,excellent,high,stable,stable,stable,10,S
2,high,low,excellent,high,stable,stable,mod-stable,10,A
3,mid,low,good,high,stable,unstable,mod-stable,15,A
4,mid,mid,excellent,high,stable,stable,stable,10,A


In [3]:
df['decision'][3]=df['decision'][4]
df[:5]

,L-CORE,L-SURF,L-02,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFPRT,decision
0,mid,low,excellent,mid,stable,stable,stable,15,A
1,mid,high,excellent,high,stable,stable,stable,10,S
2,high,low,excellent,high,stable,stable,mod-stable,10,A
3,mid,low,good,high,stable,unstable,mod-stable,15,A
4,mid,mid,excellent,high,stable,stable,stable,10,A


In [4]:
#df=df.drop([3])
#df[:5]


In [5]:
for c in ['L-CORE', 'L-SURF', 'L-02', 'L-BP', 'SURF-STBL', 'CORE-STBL', 'BP-STBL', 'decision']:
    df[c] = df[c].astype('category')

In [6]:
for i in range(len(df)):
    if (df['COMFPRT'][i]=='?'):
        df['COMFPRT'][i]=10

/home/jinhongkim/miniconda3/envs/ml_python/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [7]:
df["COMFPRT"] = df["COMFPRT"].astype("int64")

In [8]:
df_modified=pd.get_dummies(df)
df_modified[:5]

,COMFPRT,L-CORE_high,L-CORE_low,L-CORE_mid,L-SURF_high,L-SURF_low,L-SURF_mid,L-02_excellent,L-02_good,L-BP_high,...,SURF-STBL_unstable,CORE-STBL_mod-stable,CORE-STBL_stable,CORE-STBL_unstable,BP-STBL_mod-stable,BP-STBL_stable,BP-STBL_unstable,decision_A,decision_I,decision_S
0,15,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,10,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,10,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,15,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,10,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [9]:
x_data=df_modified.ix[:,0:-3].values
y_data=df_modified.ix[:,-3:].values
y_data.shape, x_data.shape

((90, 3), (90, 20))

In [10]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
x_data[:,0]=min_max_scaler.fit_transform(x_data[:,0])

x_data[:5,0]

/home/jinhongkim/miniconda3/envs/ml_python/lib/python3.4/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/jinhongkim/miniconda3/envs/ml_python/lib/python3.4/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


array([ 1. ,  0.5,  0.5,  1. ,  0.5])

In [11]:
training_idx = np.random.randint(y_data.shape[0], size=int(y_data.shape[0] * 0.8))
test_idx = np.random.randint(y_data.shape[0], size=int(y_data.shape[0] * 0.2))

x_training, x_test = x_data[training_idx,:], x_data[test_idx,:]
y_training, y_test = y_data[training_idx,:], y_data[test_idx,:]

x_training.shape, x_test.shape

((72, 20), (18, 20))

In [12]:
X = tf.placeholder("float", [None, len(x_data[0])])
Y = tf.placeholder("float", [None, len(y_data[0])])
W = tf.Variable(tf.zeros([len(x_data[0]), len(y_data[0])]))

hypothesis = tf.nn.softmax(tf.matmul(X, W))
cost=tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), reduction_indices=1))

a=tf.Variable(0.001)

optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

for step in range(5000000):
    sess.run(train, feed_dict={X:x_training, Y:y_training})
    if step % 250000 == 0:
        print (step, sess.run(cost, feed_dict={X:x_training, Y:y_training}), sess.run(W))

0 1.09755 [[  2.49074074e-04  -1.88425940e-04  -6.06481699e-05]
 [  1.34259273e-04  -7.40740870e-05  -6.01851971e-05]
 [  4.62962962e-05  -3.70370435e-05  -9.25926270e-06]
 [  2.22222225e-04  -2.08333353e-04  -1.38889127e-05]
 [  1.38888907e-04  -6.94444534e-05  -6.94444534e-05]
 [  6.94444534e-05  -8.33333470e-05   1.38888790e-05]
 [  1.94444452e-04  -1.66666694e-04  -2.77777926e-05]
 [  2.08333353e-04  -1.80555595e-04  -2.77777945e-05]
 [  1.94444452e-04  -1.38888907e-04  -5.55555744e-05]
 [  1.85185199e-04  -1.06481501e-04  -7.87037206e-05]
 [  2.77777799e-05  -1.38888909e-05  -1.38888909e-05]
 [  1.89814833e-04  -1.99074100e-04   9.25924633e-06]
 [  1.99074086e-04  -1.34259273e-04  -6.48148416e-05]
 [  2.03703705e-04  -1.85185214e-04  -1.85185363e-05]
 [  9.25925997e-06  -4.62963044e-06  -4.62963044e-06]
 [  3.75000061e-04  -2.63888913e-04  -1.11111156e-04]
 [  1.85185163e-05  -5.09259335e-05   3.24074063e-05]
 [  7.40740797e-05  -6.48148271e-05  -9.25926361e-06]
 [  1.62037046e-04

In [13]:
hypotehsis_value = sess.run(hypothesis, feed_dict={X:x_test})
result= [np.argmax(predict) == np.argmax(original_value)  for predict, original_value in zip(hypotehsis_value, y_test)]
sum(result) / len(result)

0.61111111111111116